# 学習済みWord2Vec
 [東北大学 乾・岡崎研究室](http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/)
 
 [白ヤギコーポレーション](https://github.com/shiroyagicorp/japanese-word2vec-model-builder)

In [87]:
# モデルのロード
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec

#model = KeyedVectors.load_word2vec_format("./entity_vector.model.bin", binary = True)

model = Word2Vec.load("./word2vec.model")

In [88]:
model.most_similar('課題')

[('問題', 0.7855098247528076),
 ('問題点', 0.7577674388885498),
 ('施策', 0.7432342171669006),
 ('現状', 0.7371587157249451),
 ('要因', 0.7271366119384766),
 ('解決策', 0.7200082540512085),
 ('方策', 0.7162488102912903),
 ('状況', 0.7085295915603638),
 ('成果', 0.7010855674743652),
 ('あり方', 0.6948018670082092)]

In [89]:
import MeCab
import re
def toWakatikiCleaned(doc):
    # Execute wakati / mecab-ipadic-neologd
    tagger = MeCab.Tagger("-d /usr/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse('')
    node = tagger.parseToNode(doc)
    #代名詞はずす？
    excludeList = ["数", "接尾", "非自立", "代名詞", "形容動詞語幹","固有名詞"]
    result = []
    fullresult = []
    node = node.next
    while node:
        meta = node.feature.split(",")
        if meta[0] == "名詞" and not meta[1] in excludeList :
            #print(node.feature)
            #print(node.surface)
            result.append(node.surface)
        fullresult.append(node.surface)
        node = node.next
    return fullresult,result

In [90]:
def toWakatiki(doc):
    # Execute wakati / mecab-ipadic-neologd
    tagger = MeCab.Tagger("-d /usr/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse('') # For mecab bug :<
    node = tagger.parseToNode(doc)
    result = []
    node = node.next
    while node:
        result.append(node.surface)
        node = node.next
    return result

In [91]:
def calcSimilarity(token, target,doc):
    result = 0
    try:
        result = model.similarity(token, target)
    except:
        pass
        #　追加学習で単語の追加はできない？
        #model.build_vocab(doc, update=True)
        #model.train(doc, total_examples = sum([len(wakati) for wakati in doc]), epochs = model.iter,compute_loss = False)
        #print(token)
        #return calcSimilarity(token,target,doc)
    return result

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
def calcTfIdf(docs):
    tokens = []
    for doc in docs:
        tokens.append(" ".join(toWakatiki(doc)))
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(tokens)
    #print(vectorizer.vocabulary_)
    return tfidf,vectorizer

In [93]:
import numpy as np
def doRecord(docs):
    tfidf,vectrorizer = calcTfIdf(docs)
    result = []
    for doc in docs:
        fulltokens,tokens = toWakatikiCleaned(doc)
        similarityK = []
        similarityT = []
        similarityS = []
        weight = []
        for token in tokens:
            similarityK.append(calcSimilarity(token,"課題",fulltokens))
            similarityT.append(calcSimilarity(token,"todo",fulltokens))
            similarityS.append(calcSimilarity(token,"質問",fulltokens))
            try:
                weight.append(vectrorizer.vocabulary_[token])
            except:
                weight.append(0)
        nsimilarityK = np.array(similarityK)
        nsimilarityT = np.array(similarityT)
        nsimilarityS = np.array(similarityS)
        nweight = np.array(weight)
        tmpK = ((nsimilarityK * nweight).sum() / nweight.sum())
        tmpT = ((nsimilarityT * nweight).sum() / nweight.sum())
        tmpS = ((nsimilarityS * nweight).sum() / nweight.sum())
        dic = {tmpK:"課題",tmpT:"todo",tmpS:"質問"}
        result.append(dic[max(tmpK,tmpT,tmpS)])
    print(result)
    # TODO
    # 再学習
    # 特徴量を計算　（文章単位/fulltokens）
    # 課題,アクションアイテム,QA　それぞれを　単語単位で word2vecで類似度を計算　（単語単位/ tokens）
    # 特徴量と加重平均をとる 

In [94]:
doRecord([#"Todoの中から優先してやらなければならないことを決めよう。",
"アマゾンジャパンの通販サイトで購入した任天堂の家庭用ゲーム機が届かないまま返送されたとの顧客の苦情が、インターネットの投稿サイトなどに相次ぎ寄せられたことが７日、分かった。配送トラブルの可能性がある。",
"「ほんやくコンニャク」は、見た目こそコンニャクそのものだが、これを食べると、外国語を自国語として聞き取れるようになり、話し言葉も自動的に話し相手の言語に翻訳されるため、言葉の通じない相手と会話できるようになる。",
"発表された車種リストには、同じ車種だがレースカテゴリやクラスごとに異なるマシンも含まれている。詳しくは下のビデオをご覧いただきたい。",
"その目的は、国土交通省に申請・届出した通りに生産されているかどうかを確認すること。",
"いずれにしても、新iPhoneを買うべきなのかどうか、あるいはもし買うならどちらか、逡巡している人はまだ多いのかもしれない。",
"デザインも存在感が少なく、日本の家庭にはこちらの方がはるかに適していると言えるのではないか。",
"AIが最重要技術であるとはいえ、AIの活用に関しては問題が山積した状態です。例えば、フェイクニュースの問題は、検索サービスやニュース記事サービスを提供するGoogleにとって解決すべき喫緊の課題で、AIを活用することで誤った情報を排除して質の高いものだけを選別する試みが行われています。"])

['質問', '質問', '課題', '課題', '質問', '課題', '課題']
